In [16]:
# Import necessary libraries
import pandas as pd               
import numpy as np                
import matplotlib.pyplot as plt   
import os                         
import joblib                     
import hopsworks                  
import re

# Import specific modules from scikit-learn
from sklearn.preprocessing import StandardScaler, OneHotEncoder   # For data preprocessing
from sklearn.metrics import accuracy_score                        # For evaluating model accuracy

from dotenv import load_dotenv
import os
load_dotenv()

#Connecting to hopsworks
api_key = os.environ.get('HOPSWORKS_API')
project = hopsworks.login(api_key_value=api_key)
fs = project.get_feature_store()

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/564374
Connected. Call `.close()` to terminate connection gracefully.


In [17]:
# Load and display the data from CSV to get an overview of the actual data - confirming the values
amd_df = pd.read_csv('AMD_stock_prices.csv')
print(amd_df.head())    

         date  1. open  2. high    3. low  4. close   5. volume ticker
0  2024-10-14  167.770   168.90  165.1500    165.27  31757205.0    AMD
1  2024-10-11  164.185   169.35  163.0101    167.89  42136175.0    AMD
2  2024-10-10  169.760   172.01  162.0000    164.18  75113590.0    AMD
3  2024-10-09  174.050   174.05  169.5500    171.02  33890735.0    AMD
4  2024-10-08  171.750   173.60  170.0000    172.80  34730152.0    AMD


In [18]:
amd_df

,date,1. open,2. high,3. low,4. close,5. volume,ticker
0,2024-10-14,167.770,168.90,165.1500,165.27,31757205.0,AMD
1,2024-10-11,164.185,169.35,163.0101,167.89,42136175.0,AMD
2,2024-10-10,169.760,172.01,162.0000,164.18,75113590.0,AMD
3,2024-10-09,174.050,174.05,169.5500,171.02,33890735.0,AMD
4,2024-10-08,171.750,173.60,170.0000,172.80,34730152.0,AMD
...,...,...,...,...,...,...,...
6273,1999-11-05,20.750,21.38,20.2500,21.25,2148000.0,AMD
6274,1999-11-04,21.130,21.31,19.9400,20.63,2415000.0,AMD
6275,1999-11-03,20.690,21.50,20.5600,21.31,2236000.0,AMD
6276,1999-11-02,20.310,20.81,20.3100,20.56,1898100.0,AMD


In [19]:
# Converting the "date" column to timestamp
amd_df['date'] = pd.to_datetime(amd_df['date'])

In [20]:
import re

# Function to clean column names
def clean_column_name(col):
    # Convert to lowercase
    col = col.lower()
    # Replace any invalid characters with an underscore
    col = re.sub(r'[^a-z0-9_]', '_', col)
    # If the column name doesn't start with a letter, prefix it with 'f_'
    if not re.match(r'^[a-z]', col):
        col = 'f_' + col
    return col

# Apply the function to all column names
amd_df.columns = [clean_column_name(col) for col in amd_df.columns]

print("Cleaned column names:")
print(amd_df.columns.tolist())


Cleaned column names:
['date', 'f_1__open', 'f_2__high', 'f_3__low', 'f_4__close', 'f_5__volume', 'ticker']


In [23]:
# version 8 for the new feature group
amd_fg = fs.create_feature_group(
    name="amd_stock",
    description="AMD stock dataset from Alpha Vantage",
    version=2,  
    primary_key=["ticker"],
    event_time="date",
    online_enabled=False,
)


In [24]:
amd_fg.insert(amd_df, write_options={"wait_for_job": False})

Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/564374/fs/560197/fg/1281372


Uploading Dataframe: 0.00% |          | Rows 0/6278 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: amd_stock_2_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/564374/jobs/named/amd_stock_2_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x27d012f8440>, None)